## NumPy fundamentals

Here, I'm working through several parts of the numpy docs and make sense of it. Covered: <br>
     - [Copies and views](https://numpy.org/doc/stable/user/basics.copies.html#view) <br>
     - [numpy.ndarray.base](https://numpy.org/doc/2.2/reference/generated/numpy.ndarray.base.html) <br>
     - [Internal organization of NumPy arrays](https://numpy.org/doc/stable/dev/internals.html#numpy-internals) <br>

### Copies and Views

- numpy arrays consist of metadata on the array and a data buffer
- most operations try to avoid copying if possible and create a new metadata object (a
  view) instead, but might create a copy is not possible to avoid
- operations like `reshape()` create a new view that have new metadata that apply to the
  same data buffer
- operations like `.T` create a new view with the metadata heavily modified (which leads
  to non-[contiguous](https://numpy.org/doc/stable/glossary.html#term-contiguous) arrays)

In [2]:
import numpy as np

# simple indexing creates a view
x = np.arange(9)
z = x[1:3] # slicing on 1d array creates a view

z.base 
# base attribute returns the base array this array is based on, or None (if it is a base array)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [7]:
x[1:3] = [10, 11] # changing a part
x

array([ 0, 10, 11,  3,  4,  5,  6,  7,  8])

In [8]:
z # z get's changes as well, because it is a view

array([10, 11])

In [9]:
# advanced indexing creates a copy
x = np.arange(9).reshape(3, 3)
x

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [10]:
y = x[[1, 2]] # advanced indexing on 2d array: `[1, 2]` is a list of rows
y

array([[3, 4, 5],
       [6, 7, 8]])

In [ ]:
y.base is None # this proofs a copy is returned, since `y` has no base in `x`

True

In [ ]:
y[0,0] = 99 # since this is a copy this change in y ...
y

array([[99,  4,  5],
       [ 6,  7,  8]])

In [ ]:
x # ... doesn't affect x at all

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [ ]:
# The numpy.reshape function creates a view where possible or a copy otherwise. This
# depends on whether the array is still contiguous (elements are still in order in the
# memory) or not.

x = np.arange(9).reshape(3, 3)
x.T.flags['C_CONTIGUOUS'] #  the transpose is not contiguous (at least not for C-ordered arrays)

False

In [ ]:
x.T.flags['F_CONTIGUOUS'] # but for F-ordered arrays it is ...

# I mean, sure: it's the other way around, but how can it be contiguous compared to
# before?

True

In [ ]:
# now let's check if strides are contiguous
x = np.arange(9)
y = x[::2]
y # array([0, 2, 4, 6, 8])
y.flags['C_CONTIGUOUS'] # neither?

# strange, that contradicts the docs, it seems, that states "In most cases, the strides
# can be modified to reshape the array with a view."
# (https://numpy.org/doc/stable/user/basics.copies.html#view)

False

In [ ]:
y.base # yet the base is still x, and thus y is a view ...

# so obviously those two concepts (view and continuity of memory addresses don't 100%
# overlap

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:
x = np.ones((2, 3))

#x.shape = 6 # we could do that to re-shape x into id

y = x.T  # makes the array non-contiguous and creates a view with reordered strides

y
# array([[1., 1.],
#        [1., 1.],
#        [1., 1.]])

y.base
# array([[1., 1., 1.],
#        [1., 1., 1.]])

y.shape = 6 # raises an error, since y is no longer contiguous

AttributeError: Incompatible shape for in-place modification. Use `.reshape()` to make a copy with the desired shape.

In [ ]:
# use .reshape to return a copy
z = y.reshape(-1)
z # array([1., 1., 1., 1., 1., 1.])
z.base # it still returns a view on y(!), but not on x

array([[1., 1.],
       [1., 1.],
       [1., 1.]])